In [ ]:
from datasets import load_dataset
import openai, json
key6 = ''
openai.api_key = key6

In [ ]:
data = load_dataset("deepmind/code_contests")['test'] 
#UNKNOWN_LANGUAGE (0): 0
# PYTHON (1) (solutions written in PYTHON2): 1921
# CPP (2): 1585
# PYTHON3 (3): 1147
# JAVA (4): 4342
data

In [ ]:
prompt_q = data[2]['description']
prompt_q

In [ ]:
def get_chatgpt_qa_response(prompt_text, temperature = 0.7, max_tokens=2048):
    messages = [{"role":"system", "content": "You are a helpful assistant that answers the question provided."},
                {"role":"user", "content": 'Provide me the Python3 codes for sovling the question: ' + prompt_text}]
    response = openai.ChatCompletion.create(
                model = "gpt-3.5-turbo",
                messages = messages,
                temperature = temperature,
                max_tokens = max_tokens,    
                )
    return response#['choices'][0]['message']['content']

In [ ]:
response = get_chatgpt_qa_response( prompt_text = chatgpt_python_codecontest[85]['question'] )

In [ ]:
response['choices'][0]['message']['content']

In [ ]:
with open( 'results/davinci_003_python_codecontest.jsonl_nocomment.jsonl', "r") as f:
    chatgpt_python_codecontest = [json.loads(x) for x in f.read().strip().split("\n") ]
len(chatgpt_python_codecontest)

In [ ]:
# chatgpt_python_codecontest[85]['gen_completion'] = response['choices'][0]['message']['content']

In [ ]:
idx = 14
chatgpt_python_codecontest[idx]['gen_completion']['choices'][0]['text'][len( chatgpt_python_codecontest[idx]['question'] ): ]

In [ ]:
for i,ins in enumerate(chatgpt_python_codecontest):
    if 'Solution:' not in ins['gen_completion']['choices'][0]['text'][len( ins['question'] ): ]:
        print(i)

In [ ]:
from utils import parse_code_snippet

for i,instance in enumerate( chatgpt_python_codecontest):
    pasrsed_codes = parse_code_snippet( prompt=instance['question'], raw_o=instance['gen_completion'] )
    chatgpt_python_codecontest[i]['pasrsed_codes'] = pasrsed_codes
    if pasrsed_codes.startswith('# CANNOT'):
        print(i)
    

In [ ]:
chatgpt_python_codecontest[30]

In [ ]:
# outputs = []
# for ins in chatgpt_python_codecontest:
#     outputs.append(json.dumps(ins))

# with open( 'results/gpt4_python_codecontest.jsonl', "w") as f:
#     f.write("\n".join(outputs) + "\n")

In [ ]:
HUMANEVAL_EOS = ["\nclass", "\ndef", "\n#", "\n@", "\nprint", "\nif"]
NON_CODE_EOS = ["<|endoftext|>", "\n```", "\n</s>", "<|endofmask|>"]
EOS = HUMANEVAL_EOS + NON_CODE_EOS

def find_gen_func_sig(prompt):
    func_sig = ""
    for x in prompt.splitlines():
        if x.startswith("def ") and x.endswith(":"):
            # always pick the last one, since there could pre-defined functions.
            func_sig = x
    return func_sig

def remove_eos(gen):
    min_index = 1000
    for eos in EOS:
        if eos in gen:
            min_index = min(min_index, gen.index(eos))
    return gen[:min_index]

In [ ]:
idx = 0
raw_o = chatgpt_python_codecontest[idx]['gen_completion'] # response['choices'][0]['message']['content'] #
prompt = chatgpt_python_codecontest[idx]['question']
if "```" in raw_o:
    gen = raw_o.split("```")[1].strip()
    if gen.startswith("python"):
        gen = gen[len("python") :].strip()
    if gen.startswith(prompt.strip()):
        suf = gen.split(prompt.strip())[-1]
        suf = remove_eos(suf)
        gen = prompt.strip() + suf
    elif find_gen_func_sig(prompt) == '':
        gen = gen
    elif find_gen_func_sig(prompt) in gen:
        # same function sign is in the prompt
        sig = find_gen_func_sig(prompt)
        pre, suf = gen.split(sig)[0], gen.split(sig)[-1]
        suf = remove_eos(suf)
        gen = pre + sig + suf
    else:
        gen = f"# CANNOT PARSE CODE SNIPPET\n{gen}"
else:
    # cannot really handle parse just dump to file and maybe process later.
    gen = f"# CANNOT PARSE\n{raw_o}"
gen


In [ ]:
gen = raw_o.split("```")[1].strip()

In [ ]:
def call_edit(prompt_text, temperature = 0.7, max_tokens=2048):

    response = openai.Edit.create( model = "text-davinci-edit-001",
                                   input = prompt_text,
                                   instruction="make minimal modifications to rewrite the codes and maintain the same functionality ",
                                   temperature=temperature,
                                    #max_tokens=max_tokens,
                                      )
    return response

In [ ]:
response = call_edit( prompt_text = chatgpt_python_codecontest[0]['gen_completion'] )
response

In [ ]:
idx = 0
chatgpt_python_codecontest[idx]['gen_completion']['choices'][0]['text'][len( chatgpt_python_codecontest[idx]['question'] ): ]